In [1]:
import numpy as np
import pandas as pd
import pickle


# EEGNet-specific imports
from EEGModels import EEGNet
from tensorflow.keras import utils as np_utils
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping

# PyRiemann imports
from pyriemann.estimation import XdawnCovariances
from pyriemann.tangentspace import TangentSpace
from pyriemann.utils.viz import plot_confusion_matrix

In [2]:
data_type = 'mind-wandering'

In [3]:
def load_dataset_ctrvsone_and_transfer(user_split = True, runidx=1, data_type='mind-wandering', one_class='HT'):
    X_train = None
    y_train_medid = None
    y_train_subjid = None
    X_val = None
    y_val_medid = None
    y_val_subjid = None
    X_test = None
    y_test_medid = None
    y_test_subjid = None

    dtype_names = ["HT", "SNY", "VIP", "CTR"]

    # user_split: Determines the creation of train/val/test set. In user_split, test/val users are never seen during the train. time_split randomly splits each user's chunks into train/val/test.
    if user_split:
        data_file_path = '../../iconip_data/{}/user_based_splits_with_timestamp_RUN{}.pkl'.format(data_type, runidx)
    else:
        data_file_path = '../../iconip_data/{}/time_based_splits_with_timestamp_RUN{}.pkl'.format(data_type, runidx)

    with open(data_file_path, 'rb') as f:
        all_data_splits = pickle.load(f)

        data_index_label_oneclass = dtype_names.index(one_class)
        data_index_label_ctr = dtype_names.index('CTR')
        transfer_index_labels = [x for x in list(set([0,1,2,3]).difference(set([dtype_names.index(one_class), 3]))) ]
        
        X_train = all_data_splits['train']['x']
        y_train_medid = all_data_splits['train']['y_med']
        y_train_subjid = all_data_splits['train']['y_subj']
        y_train_ts = all_data_splits['train']['y_ts']
        clf_config_subset_indices = np.where((y_train_medid == data_index_label_oneclass) | (y_train_medid == data_index_label_ctr))[0]
        X_train = X_train[(clf_config_subset_indices)]
        y_train_medid = y_train_medid[(clf_config_subset_indices)]
        y_train_subjid = y_train_subjid[clf_config_subset_indices]
        y_train_ts = y_train_ts[(clf_config_subset_indices)]

        X_val = all_data_splits['val']['x']
        y_val_medid = all_data_splits['val']['y_med']
        y_val_subjid = all_data_splits['val']['y_subj']
        y_val_ts = all_data_splits['val']['y_ts']
        clf_config_subset_indices = np.where((y_val_medid == data_index_label_oneclass) | (y_val_medid == data_index_label_ctr))[0]
        X_val = X_val[(clf_config_subset_indices)]
        y_val_medid = y_val_medid[(clf_config_subset_indices)]
        y_val_subjid = y_val_subjid[(clf_config_subset_indices)]
        y_val_ts = y_val_ts[(clf_config_subset_indices)]
        
        # Pretrain test and transfer test set
        X_test = all_data_splits['test']['x']
        y_test_medid = all_data_splits['test']['y_med']
        y_test_subjid = all_data_splits['test']['y_subj']
        y_test_ts = all_data_splits['test']['y_ts']
        
        # transfer test contains CTR from test and other two classes from the test.
        ctr_othertwo_indices = np.where((y_test_medid == data_index_label_ctr) | (y_test_medid == transfer_index_labels[0]) | (y_test_medid == transfer_index_labels[1]))[0]
        X_test_transfer = X_test[(ctr_othertwo_indices)]
        y_test_transfer_medid = y_test_medid[(ctr_othertwo_indices)]
        y_test_transfer_subjid = y_test_subjid[(ctr_othertwo_indices)]
        
        # pretrain test filter CTR and one_class from the test.
        clf_config_subset_indices = np.where((y_test_medid == data_index_label_oneclass) | (y_test_medid == data_index_label_ctr))[0]
        X_test = X_test[(clf_config_subset_indices)]
        y_test_medid = y_test_medid[(clf_config_subset_indices)]
        y_test_subjid = y_test_subjid[(clf_config_subset_indices)]
        y_test_ts = y_test_ts[(clf_config_subset_indices)]

    return X_train, y_train_medid, y_train_subjid, y_train_ts, X_val, y_val_medid, y_val_subjid, y_val_ts, X_test, y_test_medid, y_test_subjid, y_test_ts, X_test_transfer, y_test_transfer_medid, y_test_transfer_subjid

In [4]:
def train_model(user_split, med_tech_clf, data_type, one_class):
    
    kernels, chans, samples = 1, 64, 2560
    
    split_strat = 'user' if user_split else 'time'
    model_results = pd.DataFrame()

    for runidx in range(1, 6):
        model_results['{}_Train_Acc'.format(runidx)] = None
        model_results['{}_Val_Acc'.format(runidx)] = None
        model_results['{}_Test_Acc'.format(runidx)] = None
        model_results['{}_Test_Transfer_Acc'.format(runidx)] = None
        model_results['{}_best_params'.format(runidx)] = None


    for runidx in range(1, 6):
        print("RUN ID: ", runidx)
        med_tech_clf = True
        
        X_train, y_train_medid, y_train_subjid, y_train_ts, X_val, y_val_medid, y_val_subjid, y_val_ts, X_test, y_test_medid, y_test_subjid, y_test_ts, X_transfer, y_test_transfer_medid, y_test_transfer_subjid = load_dataset_ctrvsone_and_transfer(user_split=user_split, runidx=runidx, data_type=data_type, one_class=one_class)
        ## Convert data to binary classification task, i.e. meditation expert as class 0 vs control group as class 1
        y_train_medid = y_train_medid//3
        y_val_medid = y_val_medid//3
        y_test_medid = y_test_medid//3
        y_test_transfer_medid = y_test_transfer_medid//3

        best_clf_ours = None
        best_clf_val = 0

        #X_test_transfer, y_test_transfer_medid, y_test_transfer_subjid
        if med_tech_clf:
            Y_train      = y_train_medid
            Y_validate   = y_val_medid
            Y_test       = y_test_medid
            Y_transfer   = y_test_transfer_medid
            ckpt_file_suffix = '{}_med_clf'.format(split_strat)
        else:
            Y_train      = y_train_subjid
            Y_validate   = y_val_subjid
            Y_test       = y_test_subjid
            Y_transfer   = y_test_transfer_subjid
            ckpt_file_suffix = '{}_subj_clf'.format(split_strat)
        nb_classes = np.unique(Y_train).shape[0]
        print(np.unique(Y_train), ": unique class labels.")
    
        ############################# EEGNet portion ##################################
        # convert labels to one-hot encodings.
        Y_train      = np_utils.to_categorical(Y_train)
        Y_validate   = np_utils.to_categorical(Y_validate)
        Y_test       = np_utils.to_categorical(Y_test)
        Y_transfer   = np_utils.to_categorical(Y_transfer)

        # convert data to NHWC (trials, channels, samples, kernels) format. Data 
        # contains 60 channels and 151 time-points. Set the number of kernels to 1.
        X_train      = X_train.reshape(X_train.shape[0], chans, samples, kernels)
        X_val        = X_val.reshape(X_val.shape[0], chans, samples, kernels)
        X_test       = X_test.reshape(X_test.shape[0], chans, samples, kernels)
        X_transfer   = X_transfer.reshape(X_transfer.shape[0], chans, samples, kernels)

        print('X_train shape:', X_train.shape)
        print(X_train.shape[0], 'train samples')
        print(X_test.shape[0], 'test samples')
        print(X_transfer.shape[0], 'transfer samples')
        print(nb_classes, " number of classes")
    
        # configure the EEGNet-8,2,16 model with kernel length of 32 samples (other 
        # model configurations may do better, but this is a good starting point)
        model = EEGNet(nb_classes = nb_classes, Chans = chans, Samples = samples, 
                    dropoutRate = 0.5, kernLength = 128, F1 = 8, D = 2, F2 = 16, 
                    dropoutType = 'Dropout')

        # compile the model and set the optimizers
        model.compile(loss='categorical_crossentropy', optimizer='adam', 
                    metrics = ['accuracy'])

        # count number of parameters in the model
        numParams    = model.count_params()    

        # set a valid path for your system to record model checkpoints
        checkpointer = ModelCheckpoint(filepath='./ckpt/checkpoint_{}.h5'.format(ckpt_file_suffix), 
                                    verbose=1, save_best_only=True)
        
        ###############################################################################
        # if the classification task was imbalanced (significantly more trials in one
        # class versus the others) you can assign a weight to each class during 
        # optimization to balance it out. This data is approximately balanced so we 
        # don't need to do this, but is shown here for illustration/completeness. 
        ###############################################################################

        # the syntax is {class_1:weight_1, class_2:weight_2,...}. Here just setting
        # the weights all to be 1
        class_weights = {cl: 1 for cl in range(nb_classes)}

        ################################################################################
        # fit the model. Due to very small sample sizes this can get
        # pretty noisy run-to-run, but most runs should be comparable to xDAWN + 
        # Riemannian geometry classification (below)
        ################################################################################
        escallback = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=3)
        fittedModel = model.fit(X_train, Y_train, batch_size = 128, epochs = 10, 
                                verbose = 2, validation_data=(X_val, Y_validate),
                                callbacks=[checkpointer, escallback], class_weight = class_weights)

        # load optimal weights
        model.load_weights('./ckpt/checkpoint_{}.h5'.format(ckpt_file_suffix))
    
        probs       = model.predict(X_test)
        preds       = probs.argmax(axis = -1)

        probs_transfer = model.predict(X_transfer)
        preds_transfer = probs_transfer.argmax(axis = -1)
        
        # save preds on train and val set for error analysis
        preds_train_eegnet = model.predict(X_train)
        preds_train_eegnet = preds_train_eegnet.argmax(axis = -1)
        preds_val_eegnet = model.predict(X_val)
        preds_val_eegnet = preds_val_eegnet.argmax(axis = -1)
        
        
        acc_test         = np.mean(preds == Y_test.argmax(axis=-1))
        acc_transfer     = np.mean(preds_transfer == Y_transfer.argmax(axis=-1))
        print("EEGNet Classification accuracy test and transfer: %f %f" % (acc_test, acc_transfer))
        model_results.loc["EEGNet", ['{}_Train_Acc'.format(runidx), '{}_Val_Acc'.format(runidx), '{}_Test_Acc'.format(runidx), '{}_Test_Transfer_Acc'.format(runidx)]] = [preds_train_eegnet, preds_val_eegnet, acc_test, acc_transfer]
    if user_split:
        model_results.to_csv('./RESULTS_T5{}_MED_TransferPRED_{}_CTRvs{}.csv'.format('USER', data_type, one_class))
    else:
        model_results.to_csv('./RESULTS_T5_MED_TransferPRED_{}_CTRvs{}.csv'.format(data_type, one_class))

    return model_results

In [5]:
# time split
for one_class in ['HT', 'SNY', 'VIP']:
    model_results = train_model(user_split=False, med_tech_clf=True, data_type='mind-wandering', one_class=one_class)

RUN ID:  1
[0 1] : unique class labels.
X_train shape: (983, 64, 2560, 1)
983 train samples
348 test samples
511 transfer samples
2  number of classes


2022-06-23 14:58:04.865690: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/10

Epoch 1: val_loss improved from inf to 0.66239, saving model to ./ckpt/checkpoint_time_med_clf.h5
8/8 - 156s - loss: 0.6911 - accuracy: 0.5697 - val_loss: 0.6624 - val_accuracy: 0.6617 - 156s/epoch - 19s/step
Epoch 2/10

Epoch 2: val_loss improved from 0.66239 to 0.58635, saving model to ./ckpt/checkpoint_time_med_clf.h5
8/8 - 154s - loss: 0.5838 - accuracy: 0.6989 - val_loss: 0.5863 - val_accuracy: 0.7066 - 154s/epoch - 19s/step
Epoch 3/10

Epoch 3: val_loss improved from 0.58635 to 0.50709, saving model to ./ckpt/checkpoint_time_med_clf.h5
8/8 - 152s - loss: 0.4555 - accuracy: 0.7976 - val_loss: 0.5071 - val_accuracy: 0.8293 - 152s/epoch - 19s/step
Epoch 4/10

Epoch 4: val_loss improved from 0.50709 to 0.43435, saving model to ./ckpt/checkpoint_time_med_clf.h5
8/8 - 149s - loss: 0.3281 - accuracy: 0.8606 - val_loss: 0.4344 - val_accuracy: 0.8802 - 149s/epoch - 19s/step
Epoch 5/10

Epoch 5: val_loss improved from 0.43435 to 0.36971, saving model to ./ckpt/checkpoint_time_m

/home/singh_shruti/anaconda3/envs/compneuro/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3199: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return asarray(a).ndim


[0 1] : unique class labels.
X_train shape: (983, 64, 2560, 1)
983 train samples
348 test samples
511 transfer samples
2  number of classes
Epoch 1/10

Epoch 1: val_loss improved from inf to 0.65167, saving model to ./ckpt/checkpoint_time_med_clf.h5
8/8 - 198s - loss: 0.7049 - accuracy: 0.5270 - val_loss: 0.6517 - val_accuracy: 0.6048 - 198s/epoch - 25s/step
Epoch 2/10

Epoch 2: val_loss improved from 0.65167 to 0.60494, saving model to ./ckpt/checkpoint_time_med_clf.h5
8/8 - 191s - loss: 0.5841 - accuracy: 0.7060 - val_loss: 0.6049 - val_accuracy: 0.6587 - 191s/epoch - 24s/step
Epoch 3/10

Epoch 3: val_loss improved from 0.60494 to 0.53423, saving model to ./ckpt/checkpoint_time_med_clf.h5
8/8 - 180s - loss: 0.4705 - accuracy: 0.7976 - val_loss: 0.5342 - val_accuracy: 0.7066 - 180s/epoch - 23s/step
Epoch 4/10

Epoch 4: val_loss improved from 0.53423 to 0.41452, saving model to ./ckpt/checkpoint_time_med_clf.h5
8/8 - 181s - loss: 0.3035 - accuracy: 0.9166 - val_loss: 0.4145 - val_accur

...

/home/singh_shruti/anaconda3/envs/compneuro/lib/python3.8/site-packages/pandas/core/internals/blocks.py:938: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr_value = np.asarray(value)


[0 1] : unique class labels.
X_train shape: (979, 64, 2560, 1)
979 train samples
347 test samples
512 transfer samples
2  number of classes
Epoch 1/10

Epoch 1: val_loss improved from inf to 0.65958, saving model to ./ckpt/checkpoint_time_med_clf.h5
8/8 - 131s - loss: 0.6802 - accuracy: 0.5792 - val_loss: 0.6596 - val_accuracy: 0.5818 - 131s/epoch - 16s/step
Epoch 2/10

Epoch 2: val_loss improved from 0.65958 to 0.61729, saving model to ./ckpt/checkpoint_time_med_clf.h5
8/8 - 129s - loss: 0.6171 - accuracy: 0.6496 - val_loss: 0.6173 - val_accuracy: 0.6970 - 129s/epoch - 16s/step
Epoch 3/10

Epoch 3: val_loss improved from 0.61729 to 0.57698, saving model to ./ckpt/checkpoint_time_med_clf.h5
8/8 - 131s - loss: 0.5636 - accuracy: 0.6660 - val_loss: 0.5770 - val_accuracy: 0.7606 - 131s/epoch - 16s/step
Epoch 4/10

Epoch 4: val_loss improved from 0.57698 to 0.54646, saving model to ./ckpt/checkpoint_time_med_clf.h5
8/8 - 129s - loss: 0.5069 - accuracy: 0.7365 - val_loss: 0.5465 - val_accur

/home/singh_shruti/anaconda3/envs/compneuro/lib/python3.8/site-packages/pandas/core/internals/blocks.py:938: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr_value = np.asarray(value)


[0 1] : unique class labels.
X_train shape: (979, 64, 2560, 1)
979 train samples
347 test samples
512 transfer samples
2  number of classes
Epoch 1/10

Epoch 1: val_loss improved from inf to 0.67288, saving model to ./ckpt/checkpoint_time_med_clf.h5
8/8 - 134s - loss: 0.6985 - accuracy: 0.5741 - val_loss: 0.6729 - val_accuracy: 0.5667 - 134s/epoch - 17s/step
Epoch 2/10

Epoch 2: val_loss improved from 0.67288 to 0.63155, saving model to ./ckpt/checkpoint_time_med_clf.h5
8/8 - 129s - loss: 0.5814 - accuracy: 0.6691 - val_loss: 0.6315 - val_accuracy: 0.6636 - 129s/epoch - 16s/step
Epoch 3/10

Epoch 3: val_loss improved from 0.63155 to 0.59774, saving model to ./ckpt/checkpoint_time_med_clf.h5
8/8 - 129s - loss: 0.5221 - accuracy: 0.7120 - val_loss: 0.5977 - val_accuracy: 0.7758 - 129s/epoch - 16s/step
Epoch 4/10

Epoch 4: val_loss improved from 0.59774 to 0.56628, saving model to ./ckpt/checkpoint_time_med_clf.h5
8/8 - 130s - loss: 0.4577 - accuracy: 0.7630 - val_loss: 0.5663 - val_accur

/home/singh_shruti/anaconda3/envs/compneuro/lib/python3.8/site-packages/pandas/core/internals/blocks.py:938: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr_value = np.asarray(value)


In [ ]:
# User split
for one_class in ['HT', 'SNY', 'VIP']:
    model_results = train_model(user_split=True, med_tech_clf=True, data_type='mind-wandering', one_class=one_class)

RUN ID:  1
[0 1] : unique class labels.
X_train shape: (1044, 64, 2560, 1)
1044 train samples
320 test samples
469 transfer samples
2  number of classes
Epoch 1/10

Epoch 1: val_loss improved from inf to 0.67182, saving model to ./ckpt/checkpoint_user_med_clf.h5
9/9 - 141s - loss: 0.6731 - accuracy: 0.6188 - val_loss: 0.6718 - val_accuracy: 0.4252 - 141s/epoch - 16s/step
Epoch 2/10

Epoch 2: val_loss did not improve from 0.67182
9/9 - 142s - loss: 0.4538 - accuracy: 0.7998 - val_loss: 0.6794 - val_accuracy: 0.4252 - 142s/epoch - 16s/step
Epoch 3/10

Epoch 3: val_loss did not improve from 0.67182
9/9 - 145s - loss: 0.3138 - accuracy: 0.9004 - val_loss: 0.6945 - val_accuracy: 0.4153 - 145s/epoch - 16s/step
Epoch 4/10

Epoch 4: val_loss did not improve from 0.67182
9/9 - 145s - loss: 0.1518 - accuracy: 0.9713 - val_loss: 0.7436 - val_accuracy: 0.4983 - 145s/epoch - 16s/step
10/10 [==============================] - 1s 88ms/step
EEGNet Classification accuracy test and transfer: 0.478125 0.5

/home/singh_shruti/anaconda3/envs/compneuro/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3199: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return asarray(a).ndim


[0 1] : unique class labels.
X_train shape: (1029, 64, 2560, 1)
1029 train samples
317 test samples
478 transfer samples
2  number of classes
Epoch 1/10

Epoch 1: val_loss improved from inf to 0.73743, saving model to ./ckpt/checkpoint_user_med_clf.h5
9/9 - 142s - loss: 0.6584 - accuracy: 0.5753 - val_loss: 0.7374 - val_accuracy: 0.3354 - 142s/epoch - 16s/step
Epoch 2/10

Epoch 2: val_loss improved from 0.73743 to 0.72259, saving model to ./ckpt/checkpoint_user_med_clf.h5
9/9 - 141s - loss: 0.5721 - accuracy: 0.6997 - val_loss: 0.7226 - val_accuracy: 0.3824 - 141s/epoch - 16s/step
Epoch 3/10

Epoch 3: val_loss improved from 0.72259 to 0.68413, saving model to ./ckpt/checkpoint_user_med_clf.h5
9/9 - 144s - loss: 0.4889 - accuracy: 0.8037 - val_loss: 0.6841 - val_accuracy: 0.5705 - 144s/epoch - 16s/step
Epoch 4/10

Epoch 4: val_loss improved from 0.68413 to 0.64031, saving model to ./ckpt/checkpoint_user_med_clf.h5
9/9 - 142s - loss: 0.3971 - accuracy: 0.8571 - val_loss: 0.6403 - val_acc

/home/singh_shruti/anaconda3/envs/compneuro/lib/python3.8/site-packages/pandas/core/internals/blocks.py:938: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr_value = np.asarray(value)


[0 1] : unique class labels.
X_train shape: (1019, 64, 2560, 1)
1019 train samples
331 test samples
473 transfer samples
2  number of classes
Epoch 1/10

Epoch 1: val_loss improved from inf to 0.70437, saving model to ./ckpt/checkpoint_user_med_clf.h5
8/8 - 139s - loss: 0.7063 - accuracy: 0.5221 - val_loss: 0.7044 - val_accuracy: 0.5016 - 139s/epoch - 17s/step
Epoch 2/10

Epoch 2: val_loss did not improve from 0.70437
8/8 - 142s - loss: 0.5656 - accuracy: 0.7233 - val_loss: 0.7130 - val_accuracy: 0.3746 - 142s/epoch - 18s/step
Epoch 3/10

Epoch 3: val_loss did not improve from 0.70437
8/8 - 139s - loss: 0.4105 - accuracy: 0.8430 - val_loss: 0.7076 - val_accuracy: 0.4190 - 139s/epoch - 17s/step
Epoch 4/10

Epoch 4: val_loss did not improve from 0.70437
8/8 - 141s - loss: 0.2600 - accuracy: 0.9146 - val_loss: 0.7179 - val_accuracy: 0.6032 - 141s/epoch - 18s/step
10/10 [==============================] - 1s 116ms/step
EEGNet Classification accuracy test and transfer: 0.558912 0.646934
RUN 

...

[0 1] : unique class labels.
X_train shape: (1013, 64, 2560, 1)
1013 train samples
328 test samples
461 transfer samples
2  number of classes
Epoch 1/10

Epoch 1: val_loss improved from inf to 0.69885, saving model to ./ckpt/checkpoint_user_med_clf.h5
8/8 - 139s - loss: 0.6891 - accuracy: 0.5508 - val_loss: 0.6988 - val_accuracy: 0.5143 - 139s/epoch - 17s/step
Epoch 2/10

Epoch 2: val_loss did not improve from 0.69885
8/8 - 141s - loss: 0.5952 - accuracy: 0.7236 - val_loss: 0.7683 - val_accuracy: 0.2190 - 141s/epoch - 18s/step
Epoch 3/10

Epoch 3: val_loss did not improve from 0.69885
8/8 - 142s - loss: 0.4756 - accuracy: 0.8174 - val_loss: 0.8978 - val_accuracy: 0.2000 - 142s/epoch - 18s/step
Epoch 4/10

Epoch 4: val_loss did not improve from 0.69885
8/8 - 139s - loss: 0.3457 - accuracy: 0.8835 - val_loss: 0.9982 - val_accuracy: 0.1937 - 139s/epoch - 17s/step
10/10 [==============================] - 1s 91ms/step
EEGNet Classification accuracy test and transfer: 0.621951 0.698482
RUN I

/home/singh_shruti/anaconda3/envs/compneuro/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3199: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return asarray(a).ndim


[0 1] : unique class labels.
X_train shape: (1035, 64, 2560, 1)
1035 train samples
311 test samples
484 transfer samples
2  number of classes
Epoch 1/10

Epoch 1: val_loss improved from inf to 0.70894, saving model to ./ckpt/checkpoint_user_med_clf.h5
9/9 - 147s - loss: 0.6656 - accuracy: 0.5981 - val_loss: 0.7089 - val_accuracy: 0.4903 - 147s/epoch - 16s/step
Epoch 2/10

Epoch 2: val_loss did not improve from 0.70894
9/9 - 144s - loss: 0.4752 - accuracy: 0.8522 - val_loss: 0.7812 - val_accuracy: 0.6000 - 144s/epoch - 16s/step
Epoch 3/10

Epoch 3: val_loss did not improve from 0.70894
9/9 - 144s - loss: 0.2966 - accuracy: 0.9333 - val_loss: 0.8776 - val_accuracy: 0.5742 - 144s/epoch - 16s/step
Epoch 4/10

Epoch 4: val_loss did not improve from 0.70894
9/9 - 147s - loss: 0.1671 - accuracy: 0.9807 - val_loss: 0.8583 - val_accuracy: 0.6258 - 147s/epoch - 16s/step
10/10 [==============================] - 1s 91ms/step
EEGNet Classification accuracy test and transfer: 0.610932 0.665289
RUN I

/home/singh_shruti/anaconda3/envs/compneuro/lib/python3.8/site-packages/pandas/core/internals/blocks.py:938: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr_value = np.asarray(value)


[0 1] : unique class labels.
X_train shape: (1036, 64, 2560, 1)
1036 train samples
312 test samples
492 transfer samples
2  number of classes
Epoch 1/10


In [6]:
print("Done")

Done
